<a href="https://colab.research.google.com/github/mjkellner/MEC_Final/blob/main/Openpose1_7_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Openpose 1.7.0 Demo

In [1]:
from google.colab import drive
import os

# mount Google Drive
# When you mount it, you will be asked for permission, so allow it, copy the key, paste(Ctrl+V) it in the input field on the Colab side, and press Enter.
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Wed Mar 29 11:56:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |       

In [3]:
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

v1.7.0


In [4]:
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
! cmake --version

cmake version 3.25.2

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [7]:
# Rebuild the latest CMake as of 2021.03.03 (about 15 minutes)
! cd /content/drive/MyDrive/Colab_Notebooks/MEC_Final
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

Streaming output truncated to the last 5000 lines.
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/std   -I/content/cmake-3.19.6/Utilities  -c /content/cmake-3.19.6/Source/cmTest.cxx -o cmTest.o
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/std   -I/content/cmake-3.19.6/Utilities  -c /content/cmake-3.19.6/Source/cmTestGenerator.cxx -o cmTestGenerator.o
g++        -DCMAKE_BOOTSTRAP    -DCMake_HAVE_CXX_MAKE_UNIQUE=1   -I/content/cmake-3.19.6/Bootstrap.cmk   -I/content/cmake-3.19.6/Source   -I/content/cmake-3.19.6/Source/LexerParser   -I/content/cmake-3.19.6/Utilities/std   -I/content/cmake-3.19.6/Utilities  -c /content/cmake-3.19.6/Source/cmTimestamp.cxx

In [9]:

# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,198 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,322 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,068 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main a

In [10]:
# Clone Openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

Cloning into 'openpose'...
remote: Enumerating objects: 659, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 659 (delta 171), reused 162 (delta 24), pack-reused 0
Receiving objects: 100% (659/659), 46.51 MiB | 51.43 MiB/s, done.
Resolving deltas: 100% (171/171), done.
Note: switching to '8ca5c1d95a42340b323e9273654d1db98bec779c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [11]:
# Create build directory
! cd openpose && mkdir build && cd build

In [12]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake .. 

# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "11.8") 
--

In [13]:
# Openpose Building
! cd openpose/build && make -j`nproc`
! cd openpose && mkdir output

Scanning dependencies of target openpose_lib
[ 12%] Creating directories for 'openpose_lib'
[ 25%] No download step for 'openpose_lib'
[ 37%] No update step for 'openpose_lib'
[ 50%] No patch step for 'openpose_lib'
[ 62%] Performing configure step for 'openpose_lib'
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - sk

In [15]:
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --display 0  --write_video ./output/openpose.avi

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 21.318414 seconds.


In [16]:
# The result of the demo video is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "./openpose/output/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [45]:

# Google Drive root directory
root_path = "/content/drive/MyDrive/Colab_Notebooks/MEC_Final/Videos"

# Video path that you want to analyze on Google Drive (You can analyze the video on your own Google Drive by changing this value to a value you like)
file_path = 'Tiger Iron Back 1.mp4'

# Analyze video on Google Drive (The analysis result "openpose.avi" is output to the root directory of Google Drive)
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi"

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 15.315107 seconds.


In [46]:
# The analysis result video on Google Drive is re-encoded with ffmpeg and displayed. It will take some time.
! ffmpeg -i "$root_path/openpose.avi" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii'))

HTML(data)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [42]:
# If you want to analyze only the specified number of people, please specify the number of people after "number_people_max"
# The result can be confirmed on the browser by re-executing the code one level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --number_people_max 1

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 6.048493 seconds.


In [47]:

# If you also want to analyze the hands together, add "--hand"

# The result can be confirmed on the browser by re-executing the code two level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --hand

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 24.971078 seconds.


In [48]:

# The result can be confirmed on the browser by re-executing the code three level above.
! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path" --display 0  --write_video "$root_path/openpose.avi" --face

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 22.219935 seconds.
